In [ ]:
import pandas as pd
import numpy as np

import bambi as bmb
import kulprit as kpt

import arviz as az
import matplotlib.pyplot as plt

# define model data
data = pd.DataFrame({
    "y": np.random.normal(size=50),
    "g": np.random.choice(["Yes", "No"], size=50),
    "x1": np.random.normal(size=50),
    "x2": np.random.normal(size=50)
})
# define and fit model with MCMC
model = bmb.Model("y ~ x1 + x2", data, family="gaussian")
num_draws, num_chains = 50, 2
posterior = model.fit(draws=num_draws, chains=num_chains)
# build reference model object
ref_model = kpt.Projector(model, posterior)
# project the reference model to some parameter subset
params = ["x1", "x2"]
theta_perp = ref_model.project(params=params)
print(theta_perp.shape)
# visualise the projected model posterior
ref_model.plot_projection(params=params)

In [ ]:
az.plot_posterior(posterior, var_names=["Intercept", "x1", "x2"])
plt.show()